In [1]:
%cd ..

/home/max/process/dist/app_noisemon


In [2]:
from noisemon.schemas import EntityType

In [3]:
from noisemon.models.entity import EntityData, dataclass_to_model

In [4]:
from noisemon.database.database import SessionLocal

In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [6]:
sparql = SPARQLWrapper("http://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

## Querying company names

In [20]:
query = """
SELECT DISTINCT ?id ?idLabel WHERE {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    VALUES ?orgLike { 
        wd:Q93429702 
        wd:Q22687 
        wd:Q16691582 
        wd:Q4830453 
        wd:Q891723 
        wd:Q6881511
    }
    
    ?id wdt:P31 ?orgLike .
  
} LIMIT 200000
"""

In [21]:
sparql.setQuery(query)

results = sparql.query().convert()

In [22]:
print("[Q]             [COMPANY]")
for company in results["results"]["bindings"]:
    company_entity = company["id"]["value"]
    company_name = company["idLabel"]["value"]
    entity = EntityData(qid=company_entity, name=company_name, type=EntityType.ORGANIZATION)
    # print(entity)

[Q]             [COMPANY]


In [23]:
db = SessionLocal()
with db.begin():
    for company in results["results"]["bindings"]:
        company_entity = company["id"]["value"]
        company_name = company["idLabel"]["value"]
        entity = EntityData(qid=company_entity, name=company_name, type=EntityType.ORGANIZATION)
        # print(entity)
        model = dataclass_to_model(entity)
        db.merge(model)

In [24]:
db.commit()

In [25]:
db.close()